# Question 2 - Assignment I

The second problem statement supplies with Human Action Recognition dataset with 7 classes.

The task is implemented using a 3-layer Convolutional Neural Network along with 1-Fully Connected Layer for final output. Cross-Entropy Loss function is used to compute the loss and Stochastic Gradient Descent is used as optimizer along with the multi-step Learning rate decay.

The code utilizes the PyTorch Deep Learning Framework for implementing and executing the code.

### Imports

In [0]:
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

### Convolutional Neural Network Definition

This is a 3-layer CNN with each sequence comprising of a CNN layer, ReLU activation function and Max-Pooling layer.

In [0]:
class ConvolutionalNeuralNetwork(nn.Module):
    
    def __init__(self):
        
        super(ConvolutionalNeuralNetwork, self).__init__()

        #INPUT - 224*224*3
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        #INPUT - 112*112*32
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        #INPUT - 56*56*64
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        #INPUT - 28*28*128
        
        self.drop_out = nn.Dropout()
        self.fc = nn.Linear(28 * 28 * 128, 7)
    
    def forward(self, x):
        
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc(out)
        
        return out

### Train-Test Split Helper Function

This function sourced from PyTorch documentation helps in splitting the dataset into train and test. The image size is set at 224 * 224.

Change the dataset path according to your system.

In [0]:
# Train-Test Split Helper Function
def train_test_split(datadir, ratio = .25):
  
    train_transforms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
    test_transforms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
    
    train_data = datasets.ImageFolder(datadir, transform = train_transforms)
    test_data = datasets.ImageFolder(datadir, transform = test_transforms)
    
    number_train = len(train_data)
    indices = list(range(number_train))
    split = int(np.floor(ratio * number_train))
    np.random.shuffle(indices)
    
    
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    
    trainloader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=64)
    testloader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=64)
    
    return trainloader, testloader

data_dir = '/content/mydrive/My Drive/Random/HumanActionClassification'
trainloader, testloader = train_test_split(data_dir, .25)
print(trainloader.dataset.classes)

### Variable Declarations

In [0]:
num_epochs = 20
num_classes = 7
batch_size = 64
learning_rate = 0.001

model = ConvolutionalNeuralNetwork()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,10,15], gamma=0.1)

### CNN Training

In [0]:
total_step = len(trainloader)
loss_list = []
acc_list = []

for epoch in range(num_epochs):
  
    scheduler.step()
    
    for i, (images, labels) in enumerate(trainloader):
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%' .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(), (correct / total) * 100))

### Testing

In [0]:
with torch.no_grad():
    
    correct = 0
    total = 0
    
    for images, labels in testloader:
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy: {} %'.format((correct / total) * 100))